In [1]:
from typing import List, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader

import copy

from functools import reduce

In [2]:
class Net(nn.Module):
    """Convolutional Neural Network architecture.

    As described in McMahan 2017 paper :

    [Communication-Efficient Learning of Deep Networks from
    Decentralized Data] (https://arxiv.org/pdf/1602.05629.pdf)
    """

    def __init__(self, num_classes: int = 10) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 5, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, input_tensor: torch.Tensor) -> torch.Tensor:
        """Forward pass of the CNN.

        Parameters
        ----------
        x : torch.Tensor
            Input Tensor that will pass through the network

        Returns
        -------
        torch.Tensor
            The resulting Tensor after it has passed through the network
        """
        output_tensor = F.relu(self.conv1(input_tensor))
        output_tensor = self.pool(output_tensor)
        output_tensor = F.relu(self.conv2(output_tensor))
        output_tensor = self.pool(output_tensor)
        output_tensor = torch.flatten(output_tensor, 1)
        output_tensor = F.relu(self.fc1(output_tensor))
        output_tensor = self.fc2(output_tensor)
        return output_tensor

In [114]:
def mask_generator(dim: int, cohort_size: int, sparsity: int, rand_shuffle_flag: bin):
    mask_matrix = torch.zeros(dim, cohort_size)
    if dim > cohort_size/sparsity:
        for row_index in range(1, dim+1):
            column_numbers = range((sparsity*(row_index-1)), ((sparsity*row_index)-1)+1)
            column_indexes = []
            for elem in list(column_numbers):
                column_indexes.append(elem%cohort_size)
            mask_matrix[row_index-1][column_indexes] = 1
    else:
        for column_index in range(1,dim*sparsity+1):
            row_index = (column_index - 1)%dim + 1
            mask_matrix[row_index - 1][column_index - 1] = 1
    
    if rand_shuffle_flag:
        mask_matrix=mask_matrix[:,torch.randperm(mask_matrix.size()[1])]
    
    return mask_matrix
        
        

In [115]:
torch.t(mask_generator(3,15,4))

NameError: name 'rand_shuffle_flag' is not defined

In [112]:
list(range(6,1))

[]

In [28]:
mask_matrix = torch.zeros(20, 10)
mask_matrix[0] = torch.tensor([0,1,2,3,4,5,6,7,8,9])

In [33]:
mask_matrix[0][[3,4,5]] = 1

In [34]:
print(list(range(1,5)))

[1, 2, 3, 4]


In [35]:
mask_matrix[0][[3,4,5]]

tensor([1., 1., 1.])